In [1]:
import numpy as np
from pprint import pprint
from copy import deepcopy


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup
from laboneq.simple import *
from laboneq.dsl.device import DeviceSetup
from laboneq.dsl.calibration import Oscillator, SignalCalibration
from laboneq.dsl.enums import ModulationType




# descriptor = generate_descriptor(
#     #pqsc=[""], # 장비 여러개 사용시
#     shfqc_6=["DEV12256"], 
#     number_data_qubits=3,
#     multiplex=True,
#     number_multiplex=6,
#     include_cr_lines=False,
#     include_ef_lines=True,
#     get_zsync=False,  # Only set to True when using real device
#     save = True,
#     filename="1port",
#     ip_address="192.168.0.83"
# )


#descriptor
#setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
setup = DeviceSetup.from_yaml(filepath="/Users/yalgaeahn/JSAHN/qubit-experiment/examples/selectiveRIP/Descriptors/1port.yaml" , server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments


bus_ids = [f"b{i}" for i in range(3)]
bus_port = [4,5,6] #used 1,2,3 for qubit drive

for i, bus in zip(bus_port,bus_ids):
    setup.add_connections(
        setup.instruments[0].uid,
        # each bus uses its own drive:
        create_connection(
            to_signal=f"{bus}/drive",
            ports=f"SGCHANNELS/{i}/OUTPUT"
        ))

# Calibrate qubit drive/measure lines for oscillator phase increments
qubit_ids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
for qubit in qubit_ids:
    for line, frequency, mod_type in [
        ("drive", 5e9, ModulationType.HARDWARE),
        ("drive_ef", 6e9, ModulationType.HARDWARE),
        ("measure", 4e9, ModulationType.SOFTWARE),
    ]:
        logical_signal = setup.logical_signal_by_uid(f"{qubit}/{line}")
        oscillator = Oscillator(modulation_type=mod_type)
        logical_signal.calibration = SignalCalibration(
            local_oscillator=Oscillator(frequency=frequency),
            oscillator=oscillator,
        )
        if line == "measure":
            acquire_signal = setup.logical_signal_by_uid(f"{qubit}/acquire")
            acquire_signal.calibration = SignalCalibration(
                local_oscillator=Oscillator(frequency=frequency),
                oscillator=oscillator,
            )

from qpu_types.fixed_transmon import FixedTransmonQubit, FixedTransmonQubitParameters
from qpu_types.bus_cavity import BusCavity, BusCavityParameters
from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform
from helper import load_qubit_parameters, save_qubit_parameters

qubit_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
bus_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("b")]

qubits = FixedTransmonQubit.from_device_setup(
    setup, qubit_uids=qubit_uids)
buses = BusCavity.from_device_setup(
    setup, qubit_uids=bus_uids)

qpu = QPU(quantum_elements={"qubits" : qubits, "bus" : buses}, quantum_operations=[FixedTransmonOperations, BusCavityOperations])


# from laboneq.simple import workflow
# folder_store = workflow.logbook.FolderStore("./experiment_store") 
# folder_store.activate()
# #folder_store.deactivate()
# #workflow.logbook.LoggingStore().activate()
# #workflow.logbook.LogbookStore().deactivate()

import os
from datetime import datetime
def find_latest_json(folder_path):
    files = [f for f in os.listdir(folder_path)]
    timestamps = []
    for file in files:
        try:
            # Extract a timestamp assuming it's included in the filename
            timestamp_str = file.split('_', 1)[0]  # Assuming YYYYMMDDHHMMSS format
            timestamp = datetime.strptime(timestamp_str, '%Y%m%d-%H%M%S')
            timestamps.append((timestamp, file))
        except ValueError:
            print("fuck")
            continue  # Skip files that do not match the timestamp format

       # Find the most recent file
    if timestamps:
        latest_file = max(timestamps, key=lambda x: x[0])[1]
        return os.path.join(folder_path, latest_file)
    return None

qb_pars_file = find_latest_json("./qpu_parameters/")
print(f"LOADED: {qb_pars_file}")

from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
import laboneq.dsl.quantum.qpu as qpu_mod

class CombinedOperations(FixedTransmonOperations, BusCavityOperations):
    pass

qpu_mod.CombinedOperations = CombinedOperations

qpu = load(qb_pars_file)

buses = qpu.groups.bus
qubits = qpu.groups.qubits


from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store")
folder_store.activate()

fuck
LOADED: ./qpu_parameters/20260213-0350_tuneup


# CONNECTION

In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=False, do_emulation=False)
#session.disconnect()

[2026.02.16 14:33:08.521] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/JSAHN/qubit-experiment/examples/selectiveRIP/laboneq_output/log
[2026.02.16 14:33:08.523] INFO    VERSION: laboneq 25.10.3
[2026.02.16 14:33:08.524] INFO    Connecting to data server at 192.168.0.83:8004
[2026.02.16 14:33:08.576] INFO    Connected to Zurich Instruments LabOne Data Server version 25.10.1.4 at 192.168.0.83:8004
[2026.02.16 14:33:08.695] INFO    Configuring the device setup
[2026.02.16 14:33:08.751] INFO    The device setup is configured


# Google Quantum AI style calibration Stack

`Bring-up` (commission) → `Tune-up`(core calibration) → `Optimize`(fidelity push) → `Benchmark/Validate` → `Monitor/Auto-recal` 

0) 공통 전제 : Calibration = Parameter + metric snapshot
2) `Tune-up`   
    2.1) Single - Qubit gate tuning. 

    2.2) Readout tuning. 

    2.3) Two-qubit gate tuning. 


4) `Benchmark / Validate` 

	•	1Q: RB (isolated vs simultaneous). 

	•	2Q: XEB (isolated pair vs parallel) — datasheet에서도 두 케이스를 나눠 보고. 
    
    •	readout: confusion matrix / SPAM proxy. 


* 연구실에서 나한테 필요한 수준까지만 하면됨


# Tune-up

## Rabi amplitude

In [37]:
print(qubits[0].parameters.resonance_frequency_ge)
print(qubits[1].parameters.resonance_frequency_ge)
print(qubits[2].parameters.resonance_frequency_ge)

print(qubits[0].parameters.readout_resonator_frequency)
print(qubits[1].parameters.readout_resonator_frequency)
print(qubits[2].parameters.readout_resonator_frequency)

#qubits[0].parameters.drive_lo_frequency = 4.6e9

4207678769.491009
4538072409.427296
4751537000.0
7002100000.0
7590800000.0
7237100000.0


In [42]:
qubits[0].parameters.ge_drive_pulse['sigma']

0.3

In [28]:
from experiments import amplitude_rabi
q = qubits[2]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temp_pars.ge_drive_amplitude_pi = 1.0
temporary_parameters[q.uid] = temp_pars

#######################################################################
amplitudes = np.linspace(0.0, 1.0,101)
print(q.parameters.ge_drive_amplitude_pi)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(True)
options.use_cal_traces(True)
options.count(1024)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    options=options,
    temporary_parameters = temporary_parameters
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_result.tasks['analysis_workflow'].output)

0.2
DRIVE range (dBm) : 10, READOUT range (dBm) : 0
[2026.02.16 14:52:53.753] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.16 14:52:53.754] INFO      Workflow 'amplitude_rabi': execution started at 2026-02-16 05:52:53.753318Z   
[2026.02.16 14:52:53.754] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.16 14:52:53.772] INFO    Task 'temporary_qpu': started at 2026-02-16 05:52:53.772356Z
[2026.02.16 14:52:53.774] INFO    Task 'temporary_qpu': ended at 2026-02-16 05:52:53.774314Z
[2026.02.16 14:52:53.777] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2026-02-16 
[2026.02.16 14:52:53.777] INFO    05:52:53.777212Z
[2026.02.16 14:52:53.778] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2026-02-16 05:52:53.778107Z
[2026.02.16 14:52:53.779] INFO    Task 'create_experiment': started at 2026-02-16 05:52:53.779282Z
[2026.02.16 14:52:53.785] INFO    Task 'create_ex

[2026.02.16 14:52:53.792] ERROR   Task 'compile_experiment': failed at 2026-02-16 05:52:53.792551Z with: 


[2026.02.16 14:52:53.793] ERROR   LabOneQException("Undefined oscillator frequency on signal 'q2/acquire'")


[2026.02.16 14:52:53.793] ERROR   LabOneQException("Undefined oscillator frequency on signal 'q2/acquire'")


[2026.02.16 14:52:53.796] INFO    Task 'compile_experiment': ended at 2026-02-16 05:52:53.794553Z
[2026.02.16 14:52:53.796] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.16 14:52:53.797] INFO      Workflow 'amplitude_rabi': execution ended at 2026-02-16 05:52:53.796627Z     
[2026.02.16 14:52:53.798] INFO     ────────────────────────────────────────────────────────────────────────────── 


LabOneQException: Undefined oscillator frequency on signal 'q2/acquire'

## Ramsey

In [47]:
from laboneq_applications.experiments import ramsey

q = qubits[1]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temp_pars.readout_amplitude = 1.0
temporary_parameters[q.uid] = temp_pars
#######################################################################
delays = np.linspace(0,20e-6,301)
detunings = 0.4e6 #4e6,
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(True)
options.count(256)
options.use_cal_traces(True)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options,
    temporary_parameters=temporary_parameters
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)
#ramsey_result.tasks['analysis_workflow'].tasks['plot_population'].output['q0']

('measure',
 {'amplitude': 0.5,
  'length': 1.3e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2026.02.13 05:33:56.991] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 05:33:56.991] INFO      Workflow 'ramsey': execution started at 2026-02-12 20:33:56.990830Z           
[2026.02.13 05:33:56.992] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 05:33:57.006] INFO    Task 'temporary_qpu': started at 2026-02-12 20:33:57.006281Z
[2026.02.13 05:33:57.007] INFO    Task 'temporary_qpu': ended at 2026-02-12 20:33:57.007554Z
[2026.02.13 05:33:57.010] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2026-02-12 
[2026.02.13 05:33:57.010] INFO    20:33:57.010387Z
[2026.02.13 05:33:57.011] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2026-02-12 20:33:57.011559Z
[2026.02.13 05:33:57.012] INFO    Ta

In [31]:
from laboneq_applications.experiments import lifetime_measurement


###############################################################
q = qubits[1]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
# temp_pars.readout_length = 1.2e-6
# temp_pars.readout_integration_length = 1.2e-6
temporary_parameters[q.uid] = temp_pars


print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = lifetime_measurement.experiment_workflow.options()
options.update(True)
options.count(1024)
# workflow.show_fields(options)
###################################################################
t1 = lifetime_measurement.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    options=options,
    delays = np.linspace(0e-6, 20e-6, 101),
    temporary_parameters = temporary_parameters, 
)

t1_result = t1.run()

DRIVE range (dBm) : 10, READOUT range (dBm) : 0
('measure',
 {'amplitude': 0.5,
  'length': 1.3e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 1,
  'length': 5e-06,
  'pulse': {'can_compress': True, 'function': 'const'}})
[2026.02.13 03:20:02.191] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 03:20:02.192] INFO      Workflow 'lifetime_measurement': execution started at 2026-02-12              
[2026.02.13 03:20:02.192] INFO      18:20:02.191426Z                                                              
[2026.02.13 03:20:02.193] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 03:20:02.212] INFO    Task 'temporary_qpu': started at 2026-02-12 18:20:02.211858Z
[2026.02.13 03:20:02.213] INFO    Task 'temporary_qpu': ended at 2026-02-12 18:20:02.212959Z
[2026.02.13 03:20:02.218] INFO    Ta

# AMP

In [53]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[1]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars
#######################################################################
repetitions =np.arange(1,10)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(True)
options.use_cal_traces(True)
options.do_pca(False)
options.count(1024)

#print(workflow.show_fields(options))

###################################################################
error_amp_half = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters=temporary_parameters,
    options=options
)

error_amp_half_result = error_amp_half.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_half_result.tasks['analysis_workflow'].output)

temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars

#######################################################################
repetitions =np.arange(1,14) # Due to short T1, 20   
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(True)
options.use_cal_traces(True)
#options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters=temporary_parameters,
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2026.02.13 05:43:24.659] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 05:43:24.660] INFO      Workflow 'experiment_workflow_x90': execution started at 2026-02-12           
[2026.02.13 05:43:24.661] INFO      20:43:24.659513Z                                                              
[2026.02.13 05:43:24.661] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 05:43:24.674] INFO    Task 'temporary_qpu': started at 2026-02-12 20:43:24.674070Z
[2026.02.13 05:43:24.675] INFO    Task 'temporary_qpu': ended at 2026-02-12 20:43:24.675102Z
[2026.02.13 05:43:24.679] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2026-02-12 
[2026.02.13 05:43:24.679] INFO    20:43:24.679330Z
[2026.02.13 05:43:24.680] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2026-02-12 20:43:24.680262Z
[2026.02.13 05:43:24.681] INFO    Task 'create_experiment': started at 2026-02-12 20:

# DRAG

In [54]:
import importlib
from experiments import drag_q_scaling
importlib.reload(drag_q_scaling)

###############################################################
q = qubits[1]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)

temporary_parameters[q.uid] = temp_pars

print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = drag_q_scaling.experiment_workflow.options()
options.update(False)
options.count(1024)
# workflow.show_fields(options)
###################################################################
drag = drag_q_scaling.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    options=options,
    q_scalings=np.linspace(-0.05, 0.05, 11),
    mode="hybrid",
    fine_span=0.02,
    fine_points=9,
    temporary_parameters = temporary_parameters, 
)

drag_result = drag.run()

DRIVE range (dBm) : 10, READOUT range (dBm) : 0
('measure',
 {'amplitude': 0.5,
  'length': 1.3e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 1,
  'length': 5e-06,
  'pulse': {'can_compress': True, 'function': 'const'}})
[2026.02.13 05:44:27.297] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.13 05:44:27.298] INFO      Workflow 'drag_q_scaling': execution started at 2026-02-12 20:44:27.297127Z   
[2026.02.13 05:44:27.298] INFO     ────────────────────────────────────────────────────────────────────────────── 


[2026.02.13 05:44:27.315] INFO    Task 'normalize_mode': started at 2026-02-12 20:44:27.315003Z
[2026.02.13 05:44:27.316] INFO    Task 'normalize_mode': ended at 2026-02-12 20:44:27.316217Z
[2026.02.13 05:44:27.316] INFO    Task 'temporary_qpu': started at 2026-02-12 20:44:27.316776Z
[2026.02.13 05:44:27.319] INFO    Task 'temporary_qpu': ended at 2026-02-12 20:44:27.318647Z
[2026.02.13 05:44:27.321] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2026-02-12 
[2026.02.13 05:44:27.321] INFO    20:44:27.321392Z
[2026.02.13 05:44:27.322] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2026-02-12 20:44:27.322306Z
[2026.02.13 05:44:27.323] INFO    Task 'normalize_q_scalings_for_qubits': started at 2026-02-12 20:44:27.323495Z
[2026.02.13 05:44:27.324] INFO    Task 'normalize_q_scalings_for_qubits': ended at 2026-02-12 20:44:27.324221Z
[2026.02.13 05:44:27.324] INFO    Task 'create_experiment': started at 2026-02-12 20:44:27.324704Z
[2026.02.13 05:44:27.342] INFO  

In [59]:
drag_result.tasks['analysis_workflow'].output

{'old_parameter_values': {'q1': {'ge_drive_pulse.beta': 0}},
 'new_parameter_values': {'q1': {'ge_drive_pulse.beta': -0.010799953574376554+/-0.0012952818584883031}},
 'diagnostics': {'q1': {'accepted': True, 'reason': 'OK', 'method': 'xy3'}}}

In [20]:
from helper import adjust_amplitude_for_output_range, calculate_power
#adjust_amplitude_for_output_range(initial_output_dbm=0, initial_amplitude= 0.002, new_output_dbm=-5)
calculate_power(output_dbm=10, amplitude=0.6477,attenuation=[])

np.float64(8.113738970538932)

In [21]:
qubits[0].parameters.resonance_frequency_ge

4207651868.1677465

# SAVE QPU

In [34]:
from laboneq.serializers import save, load, from_dict, from_json, to_dict, to_json
import time

t = time.localtime()
timestamp = time.strftime('%Y%m%d-%H%M', t)

filename = "tuneup"
save(qpu, filename=f"./qpu_parameters/{timestamp}_{filename}")

